In [ ]:
import importlib
from dasilva_invariants import diagnostics, meshes, invariants
import glob
from joblib import delayed, Parallel
from datetime import datetime
import numpy as np
import pylab as plt
import seaborn as sns
import pyvista

importlib.reload(diagnostics)
importlib.reload(meshes)
importlib.reload(invariants)

In [ ]:
sns.set_style('darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
DPI = 500

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def lstar_vs_nlocaltimes_experiment(mesh_fname, mirror_deg, max_points, d, title):
    tasks = []

    rel_error_threshold = 0.01

    for i in range(2, max_points):
        tasks.append(delayed(_target)(
            mesh_fname, (d, 0, 0), mirror_deg, num_local_times=i,
            verbose=0, rel_error_threshold=rel_error_threshold
        ))

    par = Parallel(verbose=10000, n_jobs=-1, backend='multiprocessing')
    results = par(tasks)

    return results


def _target(mesh_fname, *args, **kwargs):
    if mesh_fname.endswith('.vtk'):
        mesh = pyvista.read(mesh_fname)
    else:
        mesh = meshes.get_lfm_hdf4_data(mesh_fname)
    try:
        return invariants.calculate_LStar(mesh, *args, **kwargs).LStar
    except invariants.DriftShellBisectionDoesntConverge as e:
        print(e)
        return np.nan
    except invariants.FieldLineTraceReturnedEmpty as e:
        print(e)
        return np.nan

# Processing

In [ ]:
mesh_main_fname = '../../data/LFM-20131002_RBSP/ElkStorm_mhd_2013-10-02T06-19-00Z.hdf' # datetime.datetime(2013, 10, 2, 6, 19)
mesh_quiet_fname = '../../data/LFM-20131002_RBSP/ElkStorm_mhd_2013-10-04T00-00-00Z.hdf' # datetime(2013, 10, 4, 0, 0)


In [ ]:
ls -l1d $HOME

In [ ]:
mesh_dip = meshes.get_dipole_mesh_on_lfm_grid(mesh_main_fname)
mesh_dip_fname = '//glade/u/home/danieldas//dipole.vtk'
mesh_dip.save(mesh_dip_fname)

In [ ]:
titles = [
          'LFM During Disturbed Period',
          'LFM During Quiet Time', 
          'Dipole on LFM Grid']
mesh_fnames = [
               mesh_main_fname,
               mesh_quiet_fname,
               mesh_dip_fname]
mirroring_mlats = [15, 30, 60]
max_points = 8
starting_distance = -6

Lstars = {}
counter = 0

for i in range(len(mesh_fnames)):
    for j in range(len(mirroring_mlats)):
        print(f'{counter + 1} out of {len(mesh_fnames)*len(mirroring_mlats)}')
        counter += 1
        Lstars[i, j] = lstar_vs_nlocaltimes_experiment(
            mesh_fnames[i], mirroring_mlats[j], max_points, d=starting_distance, title=titles[i],
        )

In [ ]:
x = np.arange(2, max_points)

data_min = np.nanmin(list(Lstars.values()))
data_max = np.nanmax(list(Lstars.values()))

#ylim = (data_min - (data_max - data_min) * .1,
#        data_max + (data_max - data_min) * .1)

ylim = (5, 7)

fig, axes = plt.subplots(len(mesh_fnames), 1, figsize=(14, 10),
                         sharex='all',
                         sharey='all')

fig.suptitle(f'Number of Required Local Times Experiment (Drift Starting at Nightside {-starting_distance} Re)', fontweight='bold')

axes = axes.flatten()
cnt = 0

for i in range(len(mesh_fnames)):
    ax = axes[cnt]
    cnt += 1

    for j in range(len(mirroring_mlats)):
        ax.plot(
            x, Lstars[i, j], 'o-',
            label=f'Mirroring MLAT {mirroring_mlats[j]}' + "$^\circ$"
        )
    ax.set_title(titles[i], fontweight='bold')
    ax.set_ylim(ylim)
    ax.set_ylabel('$L^*$ (Re)')

axes[0].legend(loc='upper center', ncol=3, facecolor='#ccc')
axes[-1].set_xlabel('Number of Local Times in Drift Shell')
fig.tight_layout()